In [ ]:
from hkube_notebook.pipeline.exec import PipelineExecutor
from hkube_notebook.pipeline.create import PipelineBuilder
from hkube_notebook.pipeline.tracker import TrackerType
from hkube_notebook.algorithm.manager import AlgorithmManager
import time
# put appropriate api-server URL
api_server = 'http://localhost:3000/api/v1'

# Build Pipeline

In [ ]:
zazaBuilder = PipelineBuilder(name='zaza', api_server_base_url=api_server)
zazaBuilder.add_node(node_name='green', alg_name='green-alg', input=["@flowInput.tata"])
zazaBuilder.add_node(node_name='yellow', alg_name='yellow-alg', input=["@green"])
zazaBuilder.add_node(node_name='black', alg_name='black-alg', input=["@yellow"])
# add unknown algorithm => error
zazaBuilder.add_node(node_name='gold', alg_name='gold-alg', input=["@black"])
zazaBuilder.get_raw()

# Execute raw pipeline from builder (webhook tracker)

In [ ]:
zazaRawExec = PipelineExecutor(raw=zazaBuilder.get_raw(), api_server_base_url=api_server)
results = zazaRawExec.exec(input={ 'tata': {} })

# Store pipeline, async execute 3 times (webhook listener)

In [ ]:
zazaBuilder.store()

In [ ]:
zazaStoredListenerExec = PipelineExecutor(name='zaza', api_server_base_url=api_server)
jobId1 = zazaStoredListenerExec.exec_async(input={})
jobId2 = zazaStoredListenerExec.exec_async(input={})
jobId2 = zazaStoredListenerExec.exec_async(input={})

In [ ]:
# run it when above jobs are still running to see their status
status_list = zazaStoredListenerExec.get_all_status()

# Async excute stored pipeline 3 times (status polling), stop one

In [ ]:
zazaStoredPollingExec = PipelineExecutor(name='zaza', api_server_base_url=api_server, tracker=TrackerType.POLLING)
# async execution
jobId1 = zazaStoredPollingExec.exec_async(input={})
jobId2 = zazaStoredPollingExec.exec_async(input={})
jobId3 = zazaStoredPollingExec.exec_async(input={})
time.sleep(4)
zazaStoredPollingExec.stop(jobId=jobId2)

In [ ]:
# get result of the first
results1 = zazaStoredPollingExec.get_results(jobId=jobId3, max_display=10)

# Execute pipeline with (too small) timeout

In [ ]:
results = zazaStoredPollingExec.exec(input={ 'tata': {} }, timeout_sec=2)

# Delete stored pipeline

In [ ]:
zazaBuilder.delete()

# Build & exec eval node pipeline, control dispaled size

In [ ]:
splitBuilder = PipelineBuilder(name='multi_result', api_server_base_url=api_server)
eval_split = {
    "code": [
    "function split(input) {",
    "return input[0].split(' ');",
    "}"]
}
splitBuilder.add_node(node_name='split', alg_name='eval-alg', input=["#@flowInput.text"],
                extra_data=eval_split)
splitBuilder.get_raw()

In [ ]:
splitExec = PipelineExecutor(raw=splitBuilder.get_raw(), api_server_base_url=api_server)
input = {
        "text": ["In mathematics and computer science, a directed acyclic graph ",
    "(DAG (About this sound listen)), is a finite directed graph with",
    " no directed cycles. That is, it consists of finitely many vertices and",
    "edges, with each edge directed from one vertex to another, ",
    "such that there is no way to start at any vertex v and follow",
    "a consistently-directed sequence of edges that eventually ",
    "loops back to v again. Equivalently, a DAG is a directed graph",
    "that has a topological ordering, a sequence of the vertices ",
    "such that every edge is directed from earlier to later in the sequence.",
    "The corresponding concept for undirected graphs is a forest, an undirected graph without ",
    "cycles. Choosing an orientation for a forest produces a special kind of directed acyclic graph",
    "called a polytree. However there are many other kinds of directed acyclic graph that are not"]
}
results = splitExec.exec(input=input, max_displayed_results=3)

In [ ]:
# execute without results display
results = splitExec.exec(input=input, max_displayed_results=0)

# Iterate over results

In [ ]:
i = 0
print("TOTAL: {} results".format(len(results)))
for item in results:
    print('ITEM {}: {}'.format(i, results[i]))
    i += 1

# Get pipelines

In [ ]:
# Get all stored pipelines
stored = PipelineExecutor.get_all_stored(api_server)
# ...now you may iterate on 'stored'...

# Get all running pipeline jobs
running = PipelineExecutor.get_running_jobs(api_server)

# Get Algorithms

In [ ]:
algs = AlgorithmManager.get_all(api_server)